In [1]:
import torch
import numpy as np
import polars as pl
import itertools
import warnings

In [2]:
warnings.filterwarnings("ignore")

In [3]:
class LabelledTensor:
    """
    Classe enveloppe pour un tenseur PyTorch avec des étiquettes explicites pour chaque dimension.

    Attributs
    ---------
    tensor : torch.Tensor
        Le tenseur brut.
    dim_labels : list[str]
        Liste ordonnée des noms des dimensions.
    index_to_label : dict[str, list[str]]
        Dictionnaire associant à chaque nom de dimension la liste de ses étiquettes.
    """

    def __init__(self, tensor: torch.Tensor, dim_labels: list[str], index_to_label: dict[str, list[str]]):
        """
        Initialise un objet LabelledTensor.

        Paramètres
        ----------
        tensor : torch.Tensor
            Le tenseur PyTorch à encapsuler.
        dim_labels : list[str]
            Les noms des dimensions du tenseur.
        index_to_label : dict[str, list[str]]
            Les étiquettes associées à chaque dimension.
        """
        self.tensor = tensor
        self.dim_labels = dim_labels
        self.index_to_label = index_to_label

    def to(self, device):
        """
        Déplace le tenseur vers le périphérique spécifié (CPU ou GPU).

        Paramètre
        ---------
        device : str
            'cpu' ou 'cuda' (ou tout autre périphérique reconnu par PyTorch).
        
        Retourne
        --------
        self : LabelledTensor
            L'objet lui-même après déplacement.
        """
        self.tensor = self.tensor.to(device)
        return self

    def __repr__(self):
        """
        Représentation courte de l'objet pour un affichage rapide.
        """
        return f"LabelledTensor(shape={self.tensor.shape}, dims={self.dim_labels})"

    def display(self, max_elements=100) -> pl.DataFrame:
        """
        Affiche une vue lisible du tenseur sous forme d'un DataFrame Polars avec les étiquettes.

        Paramètres
        ----------
        max_elements : int
            Nombre maximal d'éléments à afficher (pour éviter une surcharge visuelle).

        Retourne
        --------
        pl.DataFrame
            Un DataFrame Polars contenant les valeurs du tenseur et leurs étiquettes.
        """
        dims = self.tensor.shape
        dim_labels = self.dim_labels
        idx_labels = self.index_to_label

        total = self.tensor.numel()
        if total > max_elements:
            print(f"[INFO] Tenseur trop grand ({total} éléments). Affichage des {max_elements} premiers éléments seulement.\n")

        # Création de toutes les combinaisons possibles d’indices
        indices = list(itertools.product(*[range(d) for d in dims]))
        records = []

        for idx in indices[:max_elements]:
            labels = [idx_labels[dim_labels[i]][idx[i]] for i in range(len(idx))]
            value = self.tensor[idx].item()
            records.append((*labels, value))

        columns = dim_labels + ["valeur"]
        df = pl.DataFrame(records, schema=columns)
        
        return df

In [4]:
def create_symmetric_matrix(df: pl.DataFrame, device="cpu") -> LabelledTensor:
    """
    Construit une matrice symétrique des temps de parcours à partir d’un DataFrame triangle supérieur.

    Paramètres :
    ------------
    df : pl.DataFrame
        Doit contenir les colonnes "Idloc_start", "Idloc_end", "temps_parcours".
    device : str
        'cpu' ou 'cuda' selon l’appareil souhaité.

    Retour :
    --------
    LabelledTensor
        Matrice [i, j] symétrique, avec labels.
    """
    # Étape 1 : identifiants uniques ordonnés
    unique_locs = pl.concat([df["Idloc_start"], df["Idloc_end"]]).unique().sort()
    idx_to_id = unique_locs.to_list()
    id_to_idx = {idloc: idx for idx, idloc in enumerate(idx_to_id)}
    n = len(idx_to_id)

    # Étape 2 : conversion en indices numpy
    i_idx = df["Idloc_start"].to_numpy()
    j_idx = df["Idloc_end"].to_numpy()
    values = df["temps_parcours"].to_numpy()

    # Étape 3 : mapping des ID vers index
    i_indices = np.vectorize(id_to_idx.get)(i_idx)
    j_indices = np.vectorize(id_to_idx.get)(j_idx)

    # Étape 4 : remplissage de la matrice via vecteurs
    T = torch.full((n, n), float("inf"), device=device)
    indices = torch.tensor(np.stack([i_indices, j_indices]), device=device)
    distances = torch.tensor(values, dtype=torch.float32, device=device)

    # Triangle supérieur
    T[indices[0], indices[1]] = distances
    # Symétrie
    T[indices[1], indices[0]] = distances
    # Diagonale
    T.fill_diagonal_(0.0)

    return LabelledTensor(T, ["i", "j"], {"i": idx_to_id, "j": idx_to_id})


In [5]:
def create_population_tensor(df_pop: pl.DataFrame, idloc_order: list[str], device="cpu") -> LabelledTensor:
    """
    Crée un vecteur de population ordonné selon idloc.

    Paramètres :
    ------------
    df_pop : pl.DataFrame 
        Contient les colonnes "idloc" et "taille_population".
    idloc_order : list[str]
        Ordre des localités à respecter.
    device : str
        Appareil.

    Retour :
    --------
    LabelledTensor
        Vecteur [i] avec les tailles de population.
    """
    pop = torch.tensor(
        [df_pop.filter(pl.col("Idloc") == loc)["taille_population"][0] for loc in idloc_order],
        dtype=torch.float32,
        device=device
    )
    return LabelledTensor(pop, ["i"], {"i": idloc_order})


In [6]:
def create_infrastructure_tensor(df: pl.DataFrame, device="cpu") -> LabelledTensor:
    """
    Construit un tenseur D[i, t] représentant les infrastructures disponibles.

    Les noms des colonnes encodent le secteur (k) dans leurs deux premiers caractères.

    Paramètres :
    ------------
    df : pl.DataFrame
        Doit contenir la colonne "idloc" et des colonnes de sous-secteurs.
    device : str
        Appareil.

    Retour :
    --------
    LabelledTensor
        Tenseur [i, t] : localité × sous-secteur.
    """
    idlocs = df["idloc"].to_list()
    df_data = df.drop("idloc")
    sous_secteurs = df_data.columns

    D = torch.tensor(
        df_data.to_numpy(),
        dtype=torch.float32,
        device=device
    )

    return LabelledTensor(D, ["i", "t"], {
        "i": idlocs,
        "t": sous_secteurs
    })


In [7]:
def compute_Y(T: LabelledTensor, D: LabelledTensor) -> LabelledTensor:
    """
    Calcule le temps minimal de chaque localité i vers une autre j disposant d'une infrastructure t.

    Y[i, t] = min_j { T[i, j] | D[j, t] > 0 }

    Version sans boucle explicite, entièrement vectorisée.

    Paramètres :
    ------------
    T : LabelledTensor
        Matrice des temps de parcours [i, j].
    D : LabelledTensor
        Tenseur d'infrastructure [j, t].

    Retour :
    --------
    LabelledTensor
        Tenseur [i, t] contenant les temps minimaux vers une infrastructure.
    """
    T_tensor = T.tensor  # [i, j]
    D_tensor = D.tensor.bool()  # [j, t]

    i_size, j_size = T_tensor.shape
    t_size = D_tensor.shape[1]

    # Stocke les résultats pour chaque t
    results = []

    for t in range(t_size):
        # Indices des j où l'infrastructure t existe
        j_mask = D_tensor[:, t]  # [j]
        if j_mask.any():
            # Sous-matrice T[:, j sélectionnés]
            T_filtered = T_tensor[:, j_mask]
            min_t = T_filtered.min(dim=1).values  # [i]
        else:
            # Aucun j valide pour ce t
            min_t = torch.full((i_size,), float("inf"), device=T_tensor.device)

        results.append(min_t)

    # Concatène les résultats pour obtenir [i, t]
    Y_tensor = torch.stack(results, dim=1)  # [i, t]

    return LabelledTensor(Y_tensor, ["i", "t"], {
        "i": T.index_to_label["i"],
        "t": D.index_to_label["t"]
    })


In [8]:
def compute_Y_agg(Y: LabelledTensor, P: LabelledTensor) -> LabelledTensor:
    """
    Moyenne pondérée de Y[i, t] par la population de chaque localité i.

    Retourne Y_[t]

    Paramètres
    ----------
    Y : LabelledTensor
        Tenseur Y[i, t]
    P : LabelledTensor
        Population par localité [i]

    Retour
    ------
    LabelledTensor
        Moyenne pondérée [t]
    """
    P_vec = P.tensor.view(-1, 1)  # [i, 1]
    weighted = Y.tensor * P_vec   # [i, t]
    sum_pop = P.tensor.sum()

    Y_mean = (weighted.sum(dim=0) / sum_pop)  # [t]

    return LabelledTensor(Y_mean, ["t"], {
        "t": Y.index_to_label["t"]
    })


In [9]:
def normalize_Y(Y: LabelledTensor, Y_mean: LabelledTensor) -> LabelledTensor:
    """
    Normalise Y[i, t] en divisant chaque élément par la moyenne agrégée Ȳ[t].

    Paramètres :
    ------------
    Y : LabelledTensor
        Tenseur original [i, t].
    Y_mean : LabelledTensor
        Moyenne agrégée [t].

    Retour :
    --------
    LabelledTensor
        Tenseur normalisé [i, t].
    """
    norm_tensor = Y.tensor / Y_mean.tensor.unsqueeze(0) # [i, t] / [1, t]
    return LabelledTensor(norm_tensor, Y.dim_labels, Y.index_to_label)


In [10]:
def clamp_Y(Y: LabelledTensor, max_value=3.0) -> LabelledTensor:
    """
    Tronque les valeurs de Y[i, t] à une valeur maximale.

    Paramètres :
    ------------
    Y : LabelledTensor
        Tenseur à borner.
    max_value : float
        Valeur max autorisée.

    Retour :
    --------
    LabelledTensor
        Tenseur borné.
    """
    clamped = torch.clamp(Y.tensor, max=max_value)
    return LabelledTensor(clamped, Y.dim_labels, Y.index_to_label)


In [11]:
def compute_remoteness_tensor(Mat_dist: LabelledTensor,
                                 Mat_infra: LabelledTensor,
                                 Mat_pop: LabelledTensor,
                                 clamp_max: float = 3.0) -> LabelledTensor:
    """
    Calcule le tenseur d'accessibilité final Y[i, t] normalisé et borné.

    Étapes :
    -------
    1. Calcul des temps minimaux Y[i, t] depuis chaque localité i vers une infrastructure de type t.
    2. Agrégation pondérée par la population pour obtenir Y_mean[t].
    3. Normalisation du tenseur Y[i, t] par Y_mean[t].
    4. Borne la valeur maximale de Y[i, t] à `clamp_max`.

    Paramètres :
    ------------
    Mat_dist : LabelledTensor
        Matrice des distances [i, j].

    Mat_infra : LabelledTensor
        Tenseur d'infrastructure [i, t], booléen ou réel.

    Mat_pop : LabelledTensor
        Vecteur des populations [i].

    clamp_max : float
        Valeur maximale autorisée pour la normalisation (clipping final).

    Retour :
    --------
    LabelledTensor
        Tenseur d'accessibilité [i, t] normalisé et borné.
    """
    Mat_Y = compute_Y(Mat_dist, Mat_infra)
    Mat_Y_mean = compute_Y_agg(Mat_Y, Mat_pop)
    Mat_Y_norm = normalize_Y(Mat_Y, Mat_Y_mean)
    Mat_Y_final = clamp_Y(Mat_Y_norm, max_value=clamp_max)

    return Mat_Y_final


In [12]:
def load_all_matrices(path_dt: str,
                        path_infra: str,
                        path_pop: str,
                        device: str = "cuda") -> tuple[LabelledTensor, LabelledTensor, LabelledTensor]:
    """
    Charge et construit les matrices LabelledTensor nécessaires au calcul d'accessibilité :
    - Matrice des distances symétrique [i, j]
    - Tenseur des infrastructures [i, t]
    - Vecteur de population [i]

    Paramètres :
    ------------
    path_dt : str
        Chemin vers le fichier Parquet contenant les temps de parcours (triangle supérieur).
    path_infra : str
        Chemin vers le fichier Parquet contenant les infrastructures.
    path_pop : str
        Chemin vers le fichier Parquet contenant les populations.
    device : str
        Périphérique ("cpu" ou "cuda").

    Retour :
    --------
    tuple[LabelledTensor, LabelledTensor, LabelledTensor]
        Mat_dist  : Matrice des temps de parcours [i, j]
        Mat_infra : Tenseur des infrastructures [i, t]
        Mat_pop   : Vecteur des populations [i]
    """
    # Lecture des fichiers parquet
    df_polars = pl.read_parquet(path_dt)
    df_infra = pl.read_parquet(path_infra)
    df_pop = pl.read_parquet(path_pop)

    # Construction des matrices
    Mat_dist = create_symmetric_matrix(df_polars, device=device)
    Mat_infra = create_infrastructure_tensor(df_infra, device=device)
    Mat_pop = create_population_tensor(df_pop, df_pop["Idloc"].to_list(), device=device)

    return Mat_dist, Mat_infra, Mat_pop

In [13]:
path_dt = r"C:\Users\e_koffie\Documents\IAI_Project\SIMULATIONS\Data\dt_matrix_terrain.parquet"
path_infra = r"C:\Users\e_koffie\Documents\IAI_Project\SIMULATIONS\Data\infrastructures_matrix.parquet"
path_pop = r"C:\Users\e_koffie\Documents\IAI_Project\SIMULATIONS\Data\population_matrix.parquet"

In [14]:
Mat_dist, Mat_infra, Mat_pop = load_all_matrices(path_dt, path_infra, path_pop, device="cuda")

In [15]:
Mat_Y = compute_remoteness_tensor(Mat_dist, Mat_infra, Mat_pop, clamp_max=3.0)

In [16]:
Mat_Y.display(max_elements=10)

[INFO] Tenseur trop grand (1125064 éléments). Affichage des 10 premiers éléments seulement.



i,t,valeur
str,str,f64
"""010020201001""","""010010001""",0.295663
"""010020201001""","""010010002""",0.069482
"""010020201001""","""010020002""",0.517467
"""010020201001""","""010030001""",1.456377
"""010020201001""","""010030002""",1.645876
"""010020201001""","""010030003""",0.009953
"""010020201001""","""020010001""",1.146051
"""010020201001""","""020010002""",0.503248
"""010020201001""","""020010003""",0.059051
